In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

# Import our input dataset
charity_df = pd.read_csv('charity_data.csv')
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


Import and characterize the input data. Hint: Be sure to identify the following in your dataset:
What variable(s) are considered the target for your model? 
    Columns:IS_SUCCESSFUL
What variable(s) are considered to be the features for your model?
    All Columns Exept the ones being removed
What variable(s) are neither and should be removed from the input data?
    Columnnns:'IS_SUCCESSFUL','NAME','EIN'

In [2]:
train=charity_df.IS_SUCCESSFUL
train

0        1
1        1
2        0
3        1
4        1
        ..
34294    0
34295    0
34296    0
34297    1
34298    0
Name: IS_SUCCESSFUL, Length: 34299, dtype: int64

Using the methods described in this module, preprocess all numerical and categorical variables, as needed:
Combine rare categorical values via bucketing.
Encode categorical variables using one-hot encoding.
Standardize numerical variables using TensorFlow’s StandardScaler class.

In [3]:
Merge_df=charity_df.drop(columns=['NAME','EIN'])
Merge_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [4]:
# Generate our categorical variable list
charity_cat = Merge_df.dtypes[Merge_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
Merge_df[charity_cat].nunique()

# X_train[X_train.dtypes[X_train.dtypes== "object"].index].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(Merge_df[charity_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [6]:
# Merge one-hot encoded features and drop the originals
Merge_df = Merge_df.merge(encode_df,left_index=True, right_index=True)
Merge_df = Merge_df.drop(charity_cat,1)
Merge_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
# Remove loan status target from features data
y = Merge_df.IS_SUCCESSFUL
# X = Merge_df.(columns=
X = Merge_df.drop(columns=["IS_SUCCESSFUL",
                          'CLASSIFICATION_C6100','CLASSIFICATION_C4200','CLASSIFICATION_C2170','CLASSIFICATION_C1820',
                          'CLASSIFICATION_C1580','CLASSIFICATION_C1260','CLASSIFICATION_C1270',
'ORGANIZATION_Corporation', 'CLASSIFICATION_C1280', 'APPLICATION_TYPE_T2', 'APPLICATION_TYPE_T12'
,'CLASSIFICATION_C1240','CLASSIFICATION_C8000','CLASSIFICATION_C6000',
'SPECIAL_CONSIDERATIONS_Y','SPECIAL_CONSIDERATIONS_N',
'CLASSIFICATION_C1400','CLASSIFICATION_C2400','CLASSIFICATION_C7200',
'CLASSIFICATION_C1500','AFFILIATION_Regional','CLASSIFICATION_C7120','CLASSIFICATION_C8200',
'CLASSIFICATION_C2710','CLASSIFICATION_C1600','CLASSIFICATION_C1250','CLASSIFICATION_C1732',
'CLASSIFICATION_C3200','CLASSIFICATION_C2600','CLASSIFICATION_C2190','CLASSIFICATION_C1230',
'CLASSIFICATION_C5200','APPLICATION_TYPE_T14','CLASSIFICATION_C1720','CLASSIFICATION_C0',
'CLASSIFICATION_C2380','CLASSIFICATION_C1267','STATUS','CLASSIFICATION_C1238',
'CLASSIFICATION_C1257','CLASSIFICATION_C4100','CLASSIFICATION_C1236','CLASSIFICATION_C1235',
'USE_CASE_Other','CLASSIFICATION_C1278','AFFILIATION_Other','CLASSIFICATION_C1300','APPLICATION_TYPE_T15',
'APPLICATION_TYPE_T25','CLASSIFICATION_C1237','APPLICATION_TYPE_T29', 
                           'CLASSIFICATION_C3000','INCOME_AMT_0','INCOME_AMT_25000-99999','INCOME_AMT_1-9999',
'INCOME_AMT_100000-499999', 'INCOME_AMT_1M-5M','APPLICATION_TYPE_T8','APPLICATION_TYPE_T7',
'CLASSIFICATION_C2800','CLASSIFICATION_C2700','INCOME_AMT_10000-24999','CLASSIFICATION_C7100',
'ORGANIZATION_Co-operative','USE_CASE_CommunityServ','CLASSIFICATION_C5000','INCOME_AMT_10M-50M',
'CLASSIFICATION_C1700','INCOME_AMT_5M-10M','INCOME_AMT_50M+','CLASSIFICATION_C4000',
'AFFILIATION_Family/Parent','USE_CASE_Heathcare', 'APPLICATION_TYPE_T9','CLASSIFICATION_C1800',
'APPLICATION_TYPE_T13', 'AFFILIATION_National','CLASSIFICATION_C2300'])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=100, random_state=80)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")
rf_model.score(X_test_scaled, y_test)

 Random forest predictive accuracy: 0.701


0.7006413994169096

In [9]:
feature_names=X_train.columns
feature_names

Index(['ASK_AMT', 'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T17',
       'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3', 'APPLICATION_TYPE_T4',
       'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Independent',
       'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200', 'CLASSIFICATION_C1234',
       'CLASSIFICATION_C1245', 'CLASSIFICATION_C1246', 'CLASSIFICATION_C1248',
       'CLASSIFICATION_C1256', 'CLASSIFICATION_C1283', 'CLASSIFICATION_C1370',
       'CLASSIFICATION_C1570', 'CLASSIFICATION_C1728', 'CLASSIFICATION_C1900',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C2150',
       'CLASSIFICATION_C2500', 'CLASSIFICATION_C2561', 'CLASSIFICATION_C2570',
       'CLASSIFICATION_C3700', 'CLASSIFICATION_C4120', 'CLASSIFICATION_C4500',
       'CLASSIFICATION_C7000', 'CLASSIFICATION_C7210', 'CLASSIFICATION_C8210',
       'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Tr

In [10]:
sorted(zip(rf_model.feature_importances_, feature_names), reverse=True)

[(0.45577409239823735, 'ASK_AMT'),
 (0.14145776019782405, 'AFFILIATION_CompanySponsored'),
 (0.12643410290451138, 'AFFILIATION_Independent'),
 (0.029194859450825693, 'ORGANIZATION_Association'),
 (0.028178243644082706, 'APPLICATION_TYPE_T5'),
 (0.024017991581786767, 'APPLICATION_TYPE_T10'),
 (0.02272242662028761, 'CLASSIFICATION_C2100'),
 (0.020181776282027934, 'CLASSIFICATION_C2000'),
 (0.02016672288367124, 'ORGANIZATION_Trust'),
 (0.02013967617147194, 'APPLICATION_TYPE_T19'),
 (0.017474270421522713, 'APPLICATION_TYPE_T3'),
 (0.01734459166004503, 'APPLICATION_TYPE_T4'),
 (0.016690683490924035, 'CLASSIFICATION_C1000'),
 (0.013994492365819062, 'CLASSIFICATION_C7000'),
 (0.013027035224072758, 'APPLICATION_TYPE_T6'),
 (0.012651663108539317, 'USE_CASE_ProductDev'),
 (0.010239390884384202, 'USE_CASE_Preservation'),
 (0.00984130144934979, 'CLASSIFICATION_C1200'),
 (7.247673164710973e-05, 'CLASSIFICATION_C7210'),
 (4.797964425503132e-05, 'CLASSIFICATION_C1900'),
 (3.817099848572482e-05, 'CLAS

In [11]:
X_train_scaled_df=pd.DataFrame(X_train_scaled)
X_test_scaled_df=pd.DataFrame(X_test_scaled)

In [12]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 100
hidden_nodes_layer3 =  100
hidden_nodes_layer4 = 100

nn = tf.keras.models.Sequential()

# First input layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# first hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
# third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled_df, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled_df,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 3s 125us/sample - loss: 0.5712 - accuracy: 0.7221
Epoch 2/50
25724/25724 [==============================] - 3s 129us/sample - loss: 0.5620 - accuracy: 0.7268
Epoch 3/50
25724/25724 [==============================] - 4s 146us/sample - loss: 0.5597 - accuracy: 0.7282
Epoch 4/50
25724/25724 [==============================] - 4s 147us/sample - loss: 0.5592 - accuracy: 0.7292
Epoch 5/50
25724/25724 [==============================] - 4s 172us/sample - loss: 0.5584 - accuracy: 0.7294
Epoch 6/50
25724/25724 [==============================] - 3s 126us/sample - loss: 0.5567 - accuracy: 0.7294
Epoch 7/50
25724/25724 [==============================] - 3s 106us/sample - loss: 0.5562 - accuracy: 0.7312
Epoch 8/50
25724/25724 [==============================] - 3s 104us/sample - loss: 0.5562 - accuracy: 0.7306
Epoch 9/50
25724/25724 [==============================] - 3s 105us/sample - loss: 0.5552 - accuracy: 0.7310
Epoch

In [13]:
len(X_train_scaled)

25724